In [2]:
import scipy.io
import urllib.request
import dgl
import math
import numpy as np
import argparse
import torch
# from pypai.model import upload_model
import time
import multiprocessing
import pandas as pd
import os
# from odps import DataFrame
# from odps import options
# from pypai.io import TableWriter
# from pypai.io import TableReader
# from pypai.utils import env_utils
# from aistudio_common.utils import oss_utils

import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
from dgl.nn.functional import edge_softmax

Using backend: pytorch


In [3]:
## 读数据
path='data/JD/'

edge_data_contact = pd.read_csv(path + 'edge_data_contact.csv') # 通讯录关系
edge_data_sns = pd.read_csv(path + 'edge_data_sns.csv') # 好友关系
edge_data_transfer = pd.read_csv(path + 'edge_data_transfer.csv') # 转账关系

node_data_new = pd.read_csv(path + 'node_data_new.csv').sort_values('new_id', ascending=True)
node_data_old1 = pd.read_csv(path + 'node_data_old1.csv')
node_data_old2 = pd.read_csv(path + 'node_data_old2.csv')
node_data_old3 = pd.read_csv(path + 'node_data_old3.csv')
node_data_old4 = pd.read_csv(path + 'node_data_old4.csv')
node_data_old = pd.concat([node_data_old1,node_data_old2,node_data_old3,node_data_old4],axis=0).sort_values('new_id', ascending=True)

node_test_new = pd.read_csv(path + 'node_test_new.csv').sort_values('new_id', ascending=True)

In [4]:
# 空值填充
node_data_new[['jd_label']] = node_data_new[['jd_label']].fillna(2)
node_data_new = node_data_new.fillna(0.)

node_data_old[['jd_label']] = node_data_old[['jd_label']].fillna(2)
node_data_old = node_data_old.fillna(0.)

print('preprocess data')
edge_data_contact["user1_new_id"] = edge_data_contact["user1_new_id"].astype("int")
edge_data_contact["user2_new_id"] = edge_data_contact["user2_new_id"].astype("int")
edge_data_sns["user1_new_id"] = edge_data_sns["user1_new_id"].astype("int")
edge_data_sns["user2_new_id"] = edge_data_sns["user2_new_id"].astype("int")
edge_data_transfer["user1_new_id"] = edge_data_transfer["user1_new_id"].astype("int")
edge_data_transfer["user2_new_id"] = edge_data_transfer["user2_new_id"].astype("int")
node_data_new["new_id"] = node_data_new["new_id"].astype("int")
node_data_old["new_id"] = node_data_old["new_id"].astype("int")
node_test_new["new_id"] = node_test_new["new_id"].astype("int")

preprocess data


In [5]:
# 通讯录关系
# 1: new -> new
# 2: new -> old
# 3: old -> new
# 4: old -> old

edge_contacttype1_id_1 = edge_data_contact[edge_data_contact['edge_type'] == 1]['user1_new_id'].tolist()
edge_contacttype1_id_2 = edge_data_contact[edge_data_contact['edge_type'] == 1]['user2_new_id'].tolist()

edge_contacttype2_id_1 = edge_data_contact[edge_data_contact['edge_type'] == 2]['user1_new_id'].tolist()
edge_contacttype2_id_2 = edge_data_contact[edge_data_contact['edge_type'] == 2]['user2_new_id'].tolist()

edge_contacttype3_id_1 = edge_data_contact[edge_data_contact['edge_type'] == 3]['user1_new_id'].tolist()
edge_contacttype3_id_2 = edge_data_contact[edge_data_contact['edge_type'] == 3]['user2_new_id'].tolist()

edge_contacttype4_id_1 = edge_data_contact[edge_data_contact['edge_type'] == 4]['user1_new_id'].tolist()
edge_contacttype4_id_2 = edge_data_contact[edge_data_contact['edge_type'] == 4]['user2_new_id'].tolist()

# 好友关系
# 1: new -> new
# 2: new -> old
# 3: old -> new
# 4: old -> old
edge_snstype1_id_1 = edge_data_sns[edge_data_sns['edge_type'] == 1]['user1_new_id'].tolist()
edge_snstype1_id_2 = edge_data_sns[edge_data_sns['edge_type'] == 1]['user2_new_id'].tolist()

edge_snstype2_id_1 = edge_data_sns[edge_data_sns['edge_type'] == 2]['user1_new_id'].tolist()
edge_snstype2_id_2 = edge_data_sns[edge_data_sns['edge_type'] == 2]['user2_new_id'].tolist()

edge_snstype3_id_1 = edge_data_sns[edge_data_sns['edge_type'] == 3]['user1_new_id'].tolist()
edge_snstype3_id_2 = edge_data_sns[edge_data_sns['edge_type'] == 3]['user2_new_id'].tolist()

edge_snstype4_id_1 = edge_data_sns[edge_data_sns['edge_type'] == 4]['user1_new_id'].tolist()
edge_snstype4_id_2 = edge_data_sns[edge_data_sns['edge_type'] == 4]['user2_new_id'].tolist()

# 转账关系
# 1: new -> new
# 2: new -> old
# 3: old -> new
# 4: old -> old
edge_transfertype1_id_1 = edge_data_transfer[edge_data_transfer['edge_type'] == 1]['user1_new_id'].tolist()
edge_transfertype1_id_2 = edge_data_transfer[edge_data_transfer['edge_type'] == 1]['user2_new_id'].tolist()

edge_transfertype2_id_1 = edge_data_transfer[edge_data_transfer['edge_type'] == 2]['user1_new_id'].tolist()
edge_transfertype2_id_2 = edge_data_transfer[edge_data_transfer['edge_type'] == 2]['user2_new_id'].tolist()

edge_transfertype3_id_1 = edge_data_transfer[edge_data_transfer['edge_type'] == 3]['user1_new_id'].tolist()
edge_transfertype3_id_2 = edge_data_transfer[edge_data_transfer['edge_type'] == 3]['user2_new_id'].tolist()

edge_transfertype4_id_1 = edge_data_transfer[edge_data_transfer['edge_type'] == 4]['user1_new_id'].tolist()
edge_transfertype4_id_2 = edge_data_transfer[edge_data_transfer['edge_type'] == 4]['user2_new_id'].tolist()

In [6]:
# generate train/val/test split

pid = len(node_data_new["new_id"])
shuffle = np.random.RandomState(0).permutation(pid) #固定打乱顺序
# pid = len(shuffle)
train_idx = torch.tensor(shuffle[0:int(pid * 0.8)]).long()
val_idx = torch.tensor(shuffle[int(pid * 0.8):]).long() 

test_idx = torch.tensor(node_test_new["new_id"].tolist()).long()

In [7]:
print('building hetero-graph')
g = dgl.heterograph({
    ('N', 'r_contact_1', 'N'): (edge_contacttype1_id_1, edge_contacttype1_id_2),
    ('N', 'r_contact_2', 'O'): (edge_contacttype2_id_1, edge_contacttype2_id_2),
    ('O', 'r_contact_3', 'N'): (edge_contacttype3_id_1, edge_contacttype3_id_2),
    ('O', 'r_contact_4', 'O'): (edge_contacttype4_id_1, edge_contacttype4_id_2),
    ('N', 'r_sns_1', 'N'): (edge_snstype1_id_1, edge_snstype1_id_2),
    ('N', 'r_sns_2', 'O'): (edge_snstype2_id_1, edge_snstype2_id_2),
    ('O', 'r_sns_3', 'N'): (edge_snstype3_id_1, edge_snstype3_id_2),
    ('O', 'r_sns_4', 'O'): (edge_snstype4_id_1, edge_snstype4_id_2),
    ('N', 'r_transfer_1', 'N'): (edge_transfertype1_id_1, edge_transfertype1_id_2),
    ('N', 'r_transfer_2', 'O'): (edge_transfertype2_id_1, edge_transfertype2_id_2),
    ('O', 'r_transfer_3', 'N'): (edge_transfertype3_id_1, edge_transfertype3_id_2),
    ('O', 'r_transfer_4', 'O'): (edge_transfertype4_id_1, edge_transfertype4_id_2)
}, num_nodes_dict={'N': len(node_data_new["new_id"]), 'O': len(node_data_old["new_id"])}
)
print(g)
# g.nodes['A'].data['del_t'] = torch.tensor(node_data1['del_t'].tolist()).long()
# g.nodes['B'].data['del_t'] = torch.tensor(node_data2['del_t'].tolist()).long()
## 随便生成时间，实际应该从odps表里面读
# g.nodes['A'].data['del_t'] = torch.tensor(np.zeros(len(node_data1), dtype=np.int))
# g.nodes['B'].data['del_t'] = torch.tensor(np.zeros(len(node_data2), dtype=np.int))

# print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
g.nodes['N'].data['feat'] = torch.tensor(node_data_new.drop(['user_id', 'jd_label', 'new_id'], axis=1).values)
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
g.nodes['O'].data['feat'] = torch.tensor(node_data_old.drop(['user_id', 'jd_label', 'new_id'], axis=1).values)
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))

g.nodes['N'].data['label'] = torch.tensor(node_data_new['jd_label'].astype('int').tolist())

g.nodes['N'].data['is_label'] = torch.tensor(node_data_new['is_label'].astype('int').tolist())
g.nodes['O'].data['is_label'] = torch.tensor(node_data_old['is_label'].astype('int').tolist())

test_label = pd.merge(node_data_new[["new_id"]],node_test_new[["new_id","jd_label"]],how='left',on='new_id')
test_label[['jd_label']] = test_label[['jd_label']].fillna(0)
g.nodes['N'].data['test_label'] = torch.tensor(test_label['jd_label'].astype('int').tolist())

labels = g.nodes['N'].data['label']
print('number of train data:', len(train_idx))
print('number of positive labels:', torch.sum(labels[train_idx]==1))
print('number of negative labels:', torch.sum(labels[train_idx]==0))
print('number of unlabels:', torch.sum(labels[train_idx]==2))

print('******************************')

print('number of validation data:', len(val_idx))
print('number of positive labels:', torch.sum(labels[val_idx]==1))
print('number of negative labels:', torch.sum(labels[val_idx]==0))
print('number of unlabels:', torch.sum(labels[val_idx]==2))

node_dict = {}
edge_dict = {}

for ntype in g.ntypes:
    node_dict[ntype] = len(node_dict)
for etype in g.etypes:
    edge_dict[etype] = len(edge_dict)
    g.edges[etype].data['id'] = torch.ones(g.number_of_edges(etype), dtype=torch.long) * edge_dict[etype]
    
# n_inp_dict - 
    

building hetero-graph
Graph(num_nodes={'N': 50000, 'O': 192745},
      num_edges={('N', 'r_contact_1', 'N'): 425, ('N', 'r_contact_2', 'O'): 139397, ('N', 'r_sns_1', 'N'): 29, ('N', 'r_sns_2', 'O'): 13315, ('N', 'r_transfer_1', 'N'): 10, ('N', 'r_transfer_2', 'O'): 7290, ('O', 'r_contact_3', 'N'): 140507, ('O', 'r_contact_4', 'O'): 243072, ('O', 'r_sns_3', 'N'): 13237, ('O', 'r_sns_4', 'O'): 28926, ('O', 'r_transfer_3', 'N'): 7862, ('O', 'r_transfer_4', 'O'): 14871},
      metagraph=[('N', 'N', 'r_contact_1'), ('N', 'N', 'r_sns_1'), ('N', 'N', 'r_transfer_1'), ('N', 'O', 'r_contact_2'), ('N', 'O', 'r_sns_2'), ('N', 'O', 'r_transfer_2'), ('O', 'N', 'r_contact_3'), ('O', 'N', 'r_sns_3'), ('O', 'N', 'r_transfer_3'), ('O', 'O', 'r_contact_4'), ('O', 'O', 'r_sns_4'), ('O', 'O', 'r_transfer_4')])
2021-12-09 14:05:05
2021-12-09 14:05:05
number of train data: 40000
number of positive labels: tensor(657)
number of negative labels: tensor(1162)
number of unlabels: tensor(38181)
*****************

In [8]:
## 参数设置

device = torch.device("cuda:0")
# device = torch.device('cpu')

parser = argparse.ArgumentParser(description='Training GNN on ogbn-products benchmark')

parser.add_argument('--n_epoch', type=int, default=3)
parser.add_argument('--n_hid', type=int, default=64)
# parser.add_argument('--n_inp', type=int, default=g.nodes['N'].data['feat'].shape[-1])  
parser.add_argument('--clip', type=int, default=1.0)

parser.add_argument('--max_lr', type=float, default=0.001)
parser.add_argument('--batch_size', type=float, default=256)
parser.add_argument('--max_steps', type=int, default=400)
parser.add_argument('--val_step', type=int, default=1)


# args = parser.parse_args()
args = parser.parse_args(args=[])

node_inp_dict = dict()
for key in node_dict:
    node_inp_dict[node_dict[key]] = g.nodes[key].data['feat'].shape[-1]

In [11]:
## 定义各种方法、模型，可以跳过
### train和inference两个函数对同质图进行了改写

class HGTLayer(nn.Module):
    def __init__(self,
                 in_dim,
                 out_dim,
                 node_dict,
                 edge_dict,
                 n_heads,
                 dropout = 0.2,
                 use_norm = False,
                 out_key='',
                 is_change_fe = False):
        super(HGTLayer, self).__init__()

        self.in_dim        = in_dim
        self.out_dim       = out_dim
        self.node_dict     = node_dict
        self.edge_dict     = edge_dict
        self.num_types     = len(node_dict)
        self.num_relations = len(edge_dict)
        self.total_rel     = self.num_types * self.num_relations * self.num_types
        self.n_heads       = n_heads
        self.d_k           = out_dim // n_heads
        self.sqrt_dk       = math.sqrt(self.d_k)
        self.att           = None
        self.out_key = out_key

        self.k_linears   = nn.ModuleList()
        self.q_linears   = nn.ModuleList()
        self.v_linears   = nn.ModuleList()
        self.a_linears   = nn.ModuleList()
        self.norms       = nn.ModuleList()
        self.use_norm    = use_norm

        for t in range(self.num_types):
            self.k_linears.append(nn.Linear(in_dim,   out_dim))
            self.q_linears.append(nn.Linear(in_dim,   out_dim))
            self.v_linears.append(nn.Linear(in_dim,   out_dim))
            self.a_linears.append(nn.Linear(out_dim,  out_dim))
            if use_norm:
                self.norms.append(nn.LayerNorm(out_dim))

        self.relation_pri   = nn.Parameter(torch.ones(self.num_relations, self.n_heads))
        self.relation_att   = nn.Parameter(torch.Tensor(self.num_relations, n_heads, self.d_k, self.d_k))
        self.relation_msg   = nn.Parameter(torch.Tensor(self.num_relations, n_heads, self.d_k, self.d_k))
        self.skip           = nn.Parameter(torch.ones(self.num_types))
        self.drop           = nn.Dropout(dropout)
        self.my_alpha = torch.tensor(0.)
        
        self.is_change_fe = is_change_fe

        nn.init.xavier_uniform_(self.relation_att)
        nn.init.xavier_uniform_(self.relation_msg)

    def forward(self, G, h):
        with G.local_scope():
            node_dict, edge_dict = self.node_dict, self.edge_dict
            for srctype, etype, dsttype in G.canonical_etypes:
                sub_graph = G[srctype, etype, dsttype]

                k_linear = self.k_linears[node_dict[srctype]]
                v_linear = self.v_linears[node_dict[srctype]]
                q_linear = self.q_linears[node_dict[dsttype]]

                k = k_linear(h[srctype]).view(-1, self.n_heads, self.d_k)  ## 得到多头的key, query, value, size: (node_nums, n_heads, out_dim // n_heads)
                v = v_linear(h[srctype]).view(-1, self.n_heads, self.d_k)
                q = q_linear(h[dsttype][:G.number_of_dst_nodes(dsttype)]).view(-1, self.n_heads, self.d_k)

                e_id = self.edge_dict[etype]

                relation_att = self.relation_att[e_id]  ## size: (n_heads, out_dim // n_heads, out_dim // n_heads)
                relation_pri = self.relation_pri[e_id]
                relation_msg = self.relation_msg[e_id]

                k = torch.einsum("bij,ijk->bik", k, relation_att)
                v = torch.einsum("bij,ijk->bik", v, relation_msg)

                sub_graph.srcdata['k'] = k
                sub_graph.dstdata['q'] = q
                sub_graph.srcdata['v_%d' % e_id] = v

                sub_graph.apply_edges(fn.v_dot_u('q', 'k', 't'))  # 对q和k特征进行点乘，求kq值，准备求attention value
                attn_score = sub_graph.edata.pop('t').sum(-1) * relation_pri / self.sqrt_dk
                attn_score = edge_softmax(sub_graph, attn_score, norm_by='dst')

#                 print('attn_score >>>>>>>>>>>>>>>',attn_score.shape)
#                 print('srctype >>>>>>>>>>>>>>>',node_dict[srctype])
#                 print('etype >>>>>>>>>>>>>>>',e_id)
#                 print('dsttype >>>>>>>>>>>>>>>',node_dict[dsttype])
                sub_graph.edata['t'] = attn_score.unsqueeze(-1)   ## 得到每一种类型的attention value
            ## message passing along the eage type
            G.multi_update_all({etype : (fn.u_mul_e('v_%d' % e_id, 't', 'm'), fn.sum('m', 't')) \
                                for etype, e_id in edge_dict.items()}, cross_reducer = 'mean')
            new_h = {}
            '''
                Step 3: Target-specific Aggregation
                x = norm( W[node_type] * gelu( Agg(x) ) + x )
            '''
            
            for ntype in G.ntypes:
                n_id = node_dict[ntype]
                alpha = torch.sigmoid(self.skip[n_id])
                # print(alpha)
                t = G.dstnodes[ntype].data['t'].view(-1, self.out_dim)
                
                
                if self.is_change_fe:
                    dst_nodes = h[ntype][:G.number_of_dst_nodes(ntype)]
                    is_label = G.dstnodes[ntype].data['is_label'].view(-1,1)
#                     print('>>>>>>>>>>>>>>>>>is_label: ',is_label.shape)
#                     print('>>>>>>>>>>>>>>>>>h[ntype][:G.number_of_dst_nodes(ntype)]: ',h[ntype][:G.number_of_dst_nodes(ntype)].shape)
                    label_nodes = is_label*dst_nodes
                    unlabel_nodes = (1-is_label)*dst_nodes
                    unlabel_t = (1-is_label)*t
#                     print('unlabel_nodes: ',unlabel_nodes)
#                     print('unlabel_t: ',unlabel_t)
                    
                    unlabel_idx = (is_label == 0).nonzero()[:,0]
#                     print('is_label: ',is_label)
#                     print('unlabel_idx',unlabel_idx)
                    
                    delta = torch.abs(unlabel_t[unlabel_idx] - unlabel_nodes[unlabel_idx])
                    sorted_values,sorted_indexs = torch.sort(delta, dim=-1) #按行排序，sorted_values[:,0]得到邻居节点最相似的特征
            
                    dst_fe = torch.gather(unlabel_nodes[unlabel_idx],1,sorted_indexs[:,0].view(-1,1)) #按行取对应位置的特征：目标节点对应的特征
                    t_fe = torch.gather(t[unlabel_idx],1,sorted_indexs[:,0].view(-1,1)) #按行取对应位置的特征：目标节点对应的特征
#                     print('dst_fe: ',dst_fe)
#                     print('t_fe: ',t_fe)

                    # 替换目标节点的特征
                    # dim=1表示逐行进行列填充
                    unlabel_nodes[unlabel_idx] = unlabel_nodes[unlabel_idx].scatter(1, sorted_indexs[:,0].view(-1,1), t_fe.view(-1,1))
                    dst_nodes = label_nodes + unlabel_nodes
                    
                    # 替换邻居节点的特征
                    unlabel_t[unlabel_idx] = unlabel_t[unlabel_idx].scatter(1, sorted_indexs[:,0].view(-1,1), dst_fe.view(-1,1))
                    t = is_label*t + unlabel_t
                    
                    trans_out = self.drop(self.a_linears[n_id](F.gelu(t)))
#                     print('unlabel_nodes: ',unlabel_nodes)
#                     print('unlabel_t: ',unlabel_t)
                
                    trans_out = trans_out * alpha + dst_nodes * (1-alpha)
                
                else:
                    trans_out = self.drop(self.a_linears[n_id](F.gelu(t)))

                    trans_out = trans_out * alpha + h[ntype][:G.number_of_dst_nodes(ntype)] * (1-alpha)
                # trans_out = trans_out * self.my_alpha + h[ntype][:G.number_of_dst_nodes(ntype)] * (1-self.my_alpha)
                if self.use_norm:
                    new_h[ntype] = self.norms[n_id](trans_out)
                else:
                    new_h[ntype] = trans_out
                
            return new_h

class HGT(nn.Module):
    def __init__(self, G, node_dict, edge_dict, node_inp_dict, n_hid, n_out, n_layers, n_heads, use_norm = True, out_key='', use_t=False):
        super(HGT, self).__init__()
        self.node_dict = node_dict
        self.edge_dict = edge_dict
        
        ##kk
        self.gcs = nn.ModuleList()
        self.gcs2 = nn.ModuleList()
        ##kk
        
        self.node_inp_dict = node_inp_dict
        self.n_hid = n_hid
        self.n_out = n_out
        self.n_layers = n_layers
        self.adapt_ws  = nn.ModuleList()
        self.adapt_ws2  = nn.ModuleList()
        for t in range(len(node_dict)):
            self.adapt_ws.append(nn.Linear(self.node_inp_dict[t],   n_hid))
            self.adapt_ws2.append(nn.Linear(self.node_inp_dict[t],   n_hid))
        
        ##kk
        for _ in range(n_layers):
            self.gcs.append(HGTLayer(n_hid, n_hid, node_dict, edge_dict, n_heads, use_norm = use_norm, out_key=out_key))
            self.gcs2.append(HGTLayer(n_hid, n_hid, node_dict, edge_dict, n_heads, use_norm = use_norm, out_key=out_key, is_change_fe=True))
        ##kk
        
        #self._in = nn.Linear(n_inp,   n_hid)
        self.out = nn.Linear(n_hid, n_out)
        self.log_softmax_out = nn.LogSoftmax(dim=1)
        self.softmax_out = nn.Softmax(dim=1)
        self.use_t = use_t
#         self.RTE = RelTemporalEncoding(n_inp, max_len=367)

    def forward(self, block, out_key):
        h = {}
        h2 = {}
        for b in range(len(block)):
            if b == 0:
                h = {}
                h2 = {}
                for ntype in block[b].ntypes:   ## 这里写入模型输入
                    n_id = self.node_dict[ntype]
                    if self.use_t:
                        inp_emd = self.RTE(block[b].srcnodes[ntype].data['feat'], block[b].srcnodes[ntype].data['del_t'])
                        h[ntype] = F.gelu(self.adapt_ws[n_id](inp_emd))
                        h2[ntype] = F.gelu(self.adapt_ws2[n_id](inp_emd))
                    else:
                        try:
                            h[ntype] = F.gelu(self.adapt_ws[n_id](block[b].srcnodes[ntype].data['feat']))
                            h2[ntype] = F.gelu(self.adapt_ws2[n_id](block[b].srcnodes[ntype].data['feat']))
                        except:
                            h[ntype] = F.gelu(self.adapt_ws[n_id](block[b].srcnodes[ntype].data['feat'].float()))
                            h2[ntype] = F.gelu(self.adapt_ws2[n_id](block[b].srcnodes[ntype].data['feat'].float()))
#             print('>>>>>>>>>>>>>>>>>h: ',h['_N'].shape)
#             print('>>>>>>>>>>>>>>>>>h2: ',h2['_N'].shape)
            h = self.gcs[b](block[b], h) ## 表示有多少层HGT，层数越多，跳数越多
            h2 = self.gcs2[b](block[b], h2)
        return self.softmax_out(self.out(h[out_key])), self.softmax_out(self.out(h2[out_key]))


class RelTemporalEncoding(nn.Module):
    '''
        Implement the Temporal Encoding (Sinusoid) function.
    '''
    def __init__(self, n_hid, max_len = 367, dropout = 0.2):
        super(RelTemporalEncoding, self).__init__()
        position = torch.arange(0., max_len, 1).unsqueeze(1)
        div_term = (torch.exp(torch.arange(0, n_hid, 2) *
                             -(math.log(10000.0) / n_hid)))

        emb = nn.Embedding(max_len, n_hid)
        emb.weight.data[:, 0::2] = torch.sin(position * div_term) / math.sqrt(n_hid)
        emb.weight.data[:, 1::2] = torch.cos(position * div_term) / math.sqrt(n_hid)
        emb.requires_grad = False
        self.emb = emb
        self.lin = nn.Linear(n_hid, n_hid)

    def forward(self, x, t):
        try:
            self.emb(t)
        except:
            print(t)
        return x + self.lin(self.emb(t))   ## 注意两者的embedding维度要一样
    

def do_eval(feat):
    return np.array(eval(feat))


def eval_features(feats):
    p = multiprocessing.Pool()
    results = p.map(do_eval, feats)
    p.close()
    p.join()
    return results


def preprocess_feature(feats):
    lengths = len(feats)
    sep = 5000000
    trans_num = max(1, 1+int(lengths/sep))
    for i in range(trans_num):
        if i==0:
            if lengths>sep:
                result = eval_features(feats[i*sep:i*sep+sep])
            else:
                result = eval_features(feats)
            out = torch.tensor(result).float()
        elif i!=trans_num-1:
            result = torch.tensor(eval_features(feats[i*sep:i*sep+sep])).float()
            out = torch.cat([out, result], dim=0)
        else:
            result = torch.tensor(eval_features(feats[i*sep:])).float()
            out = torch.cat([out, result], dim=0)
    return out


def get_n_params(model):
    pp = 0
    for p in list(model.parameters()):
        nn = 1
        for s in list(p.size()):
            nn = nn * s
        pp += nn
    return pp


def build_dataloader(G, sample_list, dst_dict, batch_size):
    sampler = dgl.dataloading.neighbor.MultiLayerNeighborSampler(sample_list)
    collator = dgl.dataloading.NodeCollator(G, dst_dict, sampler)
    dataloader = torch.utils.data.DataLoader(collator.dataset, collate_fn=collator.collate, batch_size=batch_size, shuffle=True, drop_last=False)
    return dataloader


def train(model, G, dst_key, sample_list, train_dst_dict, val_dst_dict, batch_size, save_name):
    best_val_acc = torch.tensor(0).cuda()
    best_test_acc = torch.tensor(0)
    # train_ = torch.tensor(0)
    train_step = 0
    for epoch in np.arange(args.n_epoch) + 1:
        print('>>>>>>>>>>>> epoch >>>>>>>>>>>>>',epoch)
        
        dataloader = build_dataloader(G, sample_list, train_dst_dict, batch_size)
        
        for input_nodes, output_nodes, blocks in dataloader:
            blocks = [b.to(device) for b in blocks]
            #print('blocks: ',len(dataloader))
            if train_step > args.max_steps-1:
                break
            model.train()
            
            ##kk
            logits1,logits2 = model(blocks, dst_key)
            logits1 = torch.log(logits1)
            logits2 = torch.log(logits2)
            ##kk
            
            # loss = F.cross_entropy(logits, blocks[-1].dstdata['label'][dst_key]) 
            # loss = F.nll_loss(logits, blocks[-1].dstdata['label'][dst_key])  ## 绑定输入为log 概率
            
            ##kk
            is_label = blocks[-1].dstdata['is_label'][dst_key]  ##新增一列标记数据是否为有标签数据
            labels_idx = (is_label == 1).nonzero().view(-1,1)  
            unlabels_idx = (is_label == 0).nonzero().view(-1,1)
#             print('is_label: ',is_label)
#             print('******************is_label: ',is_label.shape)
#             print('******************logits1: ',logits1.shape)
#             print('******************logits1: ',logits1)
#             print('******************blocks[-1].dstdata[\'label\']: ',blocks[-1].dstdata['label'][dst_key].shape)
#             print('******************(1-is_label): ',(1-is_label).shape)
#             print('******************logits2: ',logits2.shape)
#             print('*****************(is_label*logits1).squeeze', torch.squeeze(is_label*logits1,1).shape)
#             print('labels_idx: ',labels_idx.shape,labels_idx)
#             print('logits1[labels_idx]: ',logits1[labels_idx].shape)
#             print('logits1[labels_idx]: ',torch.squeeze(logits1[labels_idx],1))
            ## 绑定输入为log 概率,同质图用这一行
#             loss = F.nll_loss(torch.squeeze(is_label*logits1,1), torch.squeeze(is_label*blocks[-1].dstdata['label'].view(-1,1),1)) + F.mse_loss(torch.squeeze((1-is_label)*logits1,1), torch.squeeze((1-is_label)*logits2,1))
            if not len(labels_idx):
                loss = F.mse_loss(torch.squeeze(logits1[unlabels_idx],1), torch.squeeze(logits2[unlabels_idx],1))
                print('#################nll_loss: None ')
                print('#################mse_loss: ',F.mse_loss(torch.squeeze(logits1[unlabels_idx],1), torch.squeeze(logits2[unlabels_idx],1)))
            if not len(unlabels_idx):
                loss = F.nll_loss(torch.squeeze(logits1[labels_idx],1), torch.squeeze(blocks[-1].dstdata['label'][dst_key][labels_idx],1))
                print('#################nll_loss: ',F.nll_loss(torch.squeeze(logits1[labels_idx],1), torch.squeeze(blocks[-1].dstdata['label'][dst_key][labels_idx],1)))
                print('#################mse_loss: None ')
            ## 异质图
            if len(labels_idx) and len(unlabels_idx):
                rate = 0.5
                loss = (1-rate)*F.nll_loss(torch.squeeze(logits1[labels_idx],1), torch.squeeze(blocks[-1].dstdata['label'][dst_key][labels_idx],1)) + rate*F.mse_loss(torch.squeeze(logits1[unlabels_idx],1), torch.squeeze(logits2[unlabels_idx],1))
                print('#################nll_loss: ',F.nll_loss(torch.squeeze(logits1[labels_idx],1), torch.squeeze(blocks[-1].dstdata['label'][dst_key][labels_idx],1)))
                print('#################mse_loss: ',F.mse_loss(torch.squeeze(logits1[unlabels_idx],1), torch.squeeze(logits2[unlabels_idx],1)))
            ##kk

            ##kk
#             is_label = blocks[-1].dstdata['is_label'][dst_key].view(-1,1)  ##新增一列标记数据是否为有标签数据
# #             print('******************is_label: ',is_label.shape)
# #             print('******************logits1: ',logits1.shape)
# #             print('******************blocks[-1].dstdata[\'label\']: ',blocks[-1].dstdata['label'].shape)
# #             print('******************(1-is_label): ',(1-is_label).shape)
# #             print('******************logits2: ',logits2.shape)
# #             print('*****************(is_label*logits1).squeeze', torch.squeeze(is_label*logits1,1).shape)
#             ## 绑定输入为log 概率,同质图用这一行
# #             loss = F.nll_loss(torch.squeeze(is_label*logits1,1), torch.squeeze(is_label*blocks[-1].dstdata['label'].view(-1,1),1)) + F.mse_loss(torch.squeeze((1-is_label)*logits1,1), torch.squeeze((1-is_label)*logits2,1))
#             ## 异质图
#             loss = F.nll_loss(torch.squeeze(is_label*logits1,1), torch.squeeze(is_label*blocks[-1].dstdata['label'][dst_key].view(-1,1),1)) + F.mse_loss(torch.squeeze((1-is_label)*logits1,1), torch.squeeze((1-is_label)*logits2,1))
#             print('#################nll_loss: ',F.nll_loss(torch.squeeze(is_label*logits1,1), torch.squeeze(is_label*blocks[-1].dstdata['label'][dst_key].view(-1,1),1)))
#             print('#################mse_loss: ',F.mse_loss(torch.squeeze((1-is_label)*logits1,1), torch.squeeze((1-is_label)*logits2,1)))
            ##kk
            
            # The loss is computed only for labeled nodes.
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), args.clip)
            optimizer.step()
            train_step += 1
            
#             for name, parms in model.named_parameters():
#                 print('-->name:', name, '-->grad_requirs:',parms.requires_grad, \
#                       ' -->grad_value:',parms.grad)

            if train_step % args.val_step == 0:
                if train_step!=1:
                    scheduler.step()
                print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
                print("train_step", train_step)
                #train_pred = logits1[:,:2].argmax(1)
                
                
                train_pred = torch.zeros(len(logits1[labels_idx]))
                score1 = logits1[labels_idx][:,0][:,1]
                pred_values,pred_indexs = torch.sort(score1, dim=-1) #按行排序，pred_indexs[:,0]得到位置排序
                train_pred[pred_indexs[:len(logits1[labels_idx])//2+1]] = 1
#                 print('pred: ',train_pred)
#                 print('label: ',blocks[-1].dstdata['label'][dst_key][labels_idx].reshape(-1))
#                 print('result: ',train_pred.cuda() == blocks[-1].dstdata['label'][dst_key][labels_idx].reshape(-1))
                
                train_acc = (train_pred.cuda() == blocks[-1].dstdata['label'][dst_key][labels_idx].reshape(-1)).float().mean()
                
                
                # train_acc = (train_pred == blocks[-1].dstdata['label']).float().mean() # 同质图用这一行
                
                val_acc_list = []
                model.eval()
                dataloader_val = build_dataloader(G, sample_list, val_dst_dict, batch_size)
                
                for input_nodes0, output_nodes0, blocks0 in dataloader_val: 
                    
                    is_val_label = blocks0[-1].dstdata['is_label'][dst_key]  ##新增一列标记数据是否为有标签数据
                    val_labels_idx = (is_val_label == 1).nonzero().view(-1,1)
                    if not len(val_labels_idx):
                        continue
                    
                    blocks0 = [b.to(device) for b in blocks0]
                    logits1,_ = model(blocks0, dst_key)  ## 做inference
                    #pred = logits1[:,:2].argmax(1)
                    
                    pred = torch.zeros(len(logits1[val_labels_idx]))
                    score1 = logits1[val_labels_idx][:,0][:,1]
                    val_values,val_indexs = torch.sort(score1, dim=-1) #按行排序，pred_indexs[:,0]得到位置排序
                    pred[val_indexs[:len(logits1[val_labels_idx])//2+1]] = 1
                    
#                     print('pred: ',pred)
#                     print('label: ',blocks0[-1].dstdata['label'][dst_key][val_labels_idx].reshape(-1))
#                     print('result: ',pred.cuda() == blocks0[-1].dstdata['label'][dst_key][val_labels_idx].reshape(-1))
                    
                    val_acc = (pred.cuda() == blocks0[-1].dstdata['label'][dst_key][val_labels_idx].reshape(-1)).float().mean()
                    # val_acc = (pred == blocks0[-1].dstdata['label']).float().mean().cuda() # 同质图用这一行
                    val_acc_list.append(val_acc)
#                     print('pred[labels_idx]: ',pred[val_labels_idx])
#                     print('val_acc》》》: ',val_acc)
                    
                
                val_acc = sum(val_acc_list)/len(val_acc_list)
#                 print('val_acc in device :',val_acc_list)
#                 print('best_val_acc in device :',len(val_acc_list).device)
                
                if best_val_acc < val_acc and val_acc-best_val_acc>0.0005:
                    best_val_acc = val_acc
                    if train_step>int(0.01*args.max_steps):
                        torch.save(model.state_dict(), 'best_' + save_name+'.pth')
                        # model_id = upload_model('best_' + save_name+".pth", name="HGT_model", model_oss_url='oss://cmps-model/323064/GNN/best_'+save_name+'.pth')
                        # model_id = upload_model('best_' + save_name+".pth", name="HGT_model")

                        print('save best model in step ', train_step)
                
#                 print('loss.grad >>>>>>>>>>> ',loss.grad)
                print('step: %d LR: %.5f train_Loss %.4f, train_acc %.4f, Val Acc %.4f (Best %.4f)' % (
                    train_step,
                    optimizer.param_groups[0]['lr'],
                    loss.item(),
                    train_acc.item(),
                    val_acc.item(),
                    best_val_acc.item(),
                ))


def inference(model, G, dst_key, sample_list, dst_dict, batch_size, write_odps_table_name, raw_label='', project='ant_prot_dev'):
    print("strat to inference")
    print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
    model.eval()
    dataloader = build_dataloader(G, sample_list, dst_dict, batch_size)  # 此处batch_size可以调大些加速inference
    infer_step = 0
    for input_nodes, output_nodes, blocks in dataloader: 
        blocks = [b.to(device) for b in blocks]
        logits,_ = model(blocks, dst_key)  # 做inference, 不需要再求log
        if raw_label!='':
            label = G.nodes[dst_key].data[raw_label][output_nodes[dst_key]].reshape([-1, 1])
            # label = G.nodes[dst_key].data[raw_label][output_nodes].reshape([-1, 1]) # 同质图用这一行
            
        else:
            label = torch.tensor([-1]*len(logits)).reshape([-1, 1])
        # print(label)

        nodes = output_nodes[dst_key].reshape([-1, 1])
        # nodes = output_nodes.reshape([-1, 1]) # 同质图这一行
        nodes = nodes.cuda()
        label = label.cuda()
        logits = logits.cuda()
        
        # alipay_id = blocks[-1].dstnodes['A'].data['alipay_id'].reshape([-1, 1]).cuda()
        if infer_step==0:
            infer_data = torch.cat([nodes, label, logits[:,:2]], dim=1).cuda()
#             label_data = torch.cat(label).cuda()
#             pred_data = torch.cat(logits[:,:2].argmax(1)).cuda()
        else:
            infer_data0 = torch.cat([nodes, label, logits[:,:2]], dim=1).cuda()
            infer_data = torch.cat([infer_data, infer_data0], dim=0)
#             label_data = torch.cat([label_data,label],dim=0)
#             pred_data = torch.cat(pred_data,[logits[:,:2].argmax(1)],dim=0)
        infer_step+=1

        if infer_step%50==0:
            print("infer_step:", infer_step)
            print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))


    infer_data = infer_data.cpu().detach().numpy()
    # writer = TableWriter(table=project+'.'+write_odps_table_name, drop_if_exists=True)   ## 写到odps表中
    ## writer = TableWriter(table=project+'.'+write_odps_table_name, drop_if_exists=True, partition="dt='20210808'")
    columns = ['id', 'label']+['score'+str(i) for i in range(len(logits[0][:2]))]
    pd_df = pd.DataFrame(infer_data, columns=columns, dtype=float)
    pd_df[['id','label','score1']].to_csv('pred_df.csv')
    # writer.to_table(pd_df)
    pred_data = pd_df[['score0','score1']].values.argmax(1)
    print('label>>>>>',len(pd_df[['label']].values))
    print('logits>>>>>',len(pred_data))
#     print('***********',pd_df[['label']].values == pred_data)
#     print('***********',np.array(pd_df[['label']].values == pred_data))
    pred_acc = np.mean(pd_df[['label']].values == pred_data)
    print('pred_acc: ',pred_acc)
    print('inference succeed')
    print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
    return pd_df

In [12]:
# 训练模型
model = HGT(g,
            node_dict, edge_dict,
            node_inp_dict=node_inp_dict,
            n_hid=args.n_hid,
#             n_out=labels.max().item() + 1,
            n_out=2,
            n_layers=3,
            n_heads=8,
            use_norm=True,
            out_key='N',
            use_t=False).to(device)

print(args.n_epoch)
# print(args.n_hid)

optimizer = torch.optim.AdamW(model.parameters())
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, total_steps=int(1+args.max_steps/args.val_step), max_lr=args.max_lr, div_factor=25.)
print('Training HGT with #param: %d' % (get_n_params(model)))
sample_list = [{('N', 'r_contact_1', 'N'): 10,
                ('N', 'r_contact_2', 'O'): 10,
                ('O', 'r_contact_3', 'N'): 10,
                ('O', 'r_contact_4', 'O'): 10,
                ('N', 'r_sns_1', 'N'): 10,
                ('N', 'r_sns_2', 'O'): 10,
                ('O', 'r_sns_3', 'N'): 10,
                ('O', 'r_sns_4', 'O'): 10,
                ('N', 'r_transfer_1', 'N'): 10,
                ('N', 'r_transfer_2', 'O'): 10,
                ('O', 'r_transfer_3', 'N'): 10,
                ('O', 'r_transfer_4', 'O'): 10}, 
               
               {('N', 'r_contact_1', 'N'): 10,
                ('N', 'r_contact_2', 'O'): 10,
                ('O', 'r_contact_3', 'N'): 10,
                ('O', 'r_contact_4', 'O'): 10,
                ('N', 'r_sns_1', 'N'): 10,
                ('N', 'r_sns_2', 'O'): 10,
                ('O', 'r_sns_3', 'N'): 10,
                ('O', 'r_sns_4', 'O'): 10,
                ('N', 'r_transfer_1', 'N'): 10,
                ('N', 'r_transfer_2', 'O'): 10,
                ('O', 'r_transfer_3', 'N'): 10,
                ('O', 'r_transfer_4', 'O'): 10}, ] # 两层模型，需要定义每一层聚合neighbor的数量
train_dst_dict = {'N': train_idx}
val_dst_dict = {'N': val_idx}
save_name = 'HGT_model_221530_jd_semi'
train(model, g, 'N', sample_list, train_dst_dict, val_dst_dict, args.batch_size, save_name)
torch.save(model.state_dict(), save_name+'.pth')
# model_id = upload_model(save_name+".pth", name="HGT_model")

3
Training HGT with #param: 293326
>>>>>>>>>>>> epoch >>>>>>>>>>>>> 1
#################nll_loss:  tensor(0.6890, device='cuda:0', grad_fn=<NllLossBackward>)
#################mse_loss:  tensor(0.1260, device='cuda:0', grad_fn=<MseLossBackward>)
2021-12-09 14:05:45
train_step 1
step: 1 LR: 0.00004 train_Loss 0.4075, train_acc 0.4615, Val Acc 0.4902 (Best 0.4902)
#################nll_loss:  tensor(0.7244, device='cuda:0', grad_fn=<NllLossBackward>)
#################mse_loss:  tensor(0.1080, device='cuda:0', grad_fn=<MseLossBackward>)
2021-12-09 14:05:51
train_step 2
step: 2 LR: 0.00004 train_Loss 0.4162, train_acc 0.5556, Val Acc 0.5056 (Best 0.5056)
#################nll_loss:  tensor(0.7287, device='cuda:0', grad_fn=<NllLossBackward>)
#################mse_loss:  tensor(0.1248, device='cuda:0', grad_fn=<MseLossBackward>)
2021-12-09 14:05:57
train_step 3
step: 3 LR: 0.00004 train_Loss 0.4268, train_acc 0.4444, Val Acc 0.4851 (Best 0.5056)
#################nll_loss:  tensor(0.7189, device='

In [13]:
# 做inference
# sample_list = [{('B', 'r1', 'A'): 10, ('A', 'r2', 'A'): 10, ('A', 'r3', 'B'): 10}, {('B', 'r1', 'A'): 10, ('A', 'r2', 'A'): 10, ('A', 'r3', 'B'): 10}]
model = HGT(g,
            node_dict, edge_dict,
            node_inp_dict=node_inp_dict,
            n_hid=args.n_hid,
            # n_out=labels.max().item() + 1,
            n_out=2,
            n_layers=3,
            n_heads=8,
            use_norm=True,
            out_key='N',
            use_t=False).to(device)

sample_list = [{('N', 'r_contact_1', 'N'): 10,
                ('N', 'r_contact_2', 'O'): 10,
                ('O', 'r_contact_3', 'N'): 10,
                ('O', 'r_contact_4', 'O'): 10,
                ('N', 'r_sns_1', 'N'): 10,
                ('N', 'r_sns_2', 'O'): 10,
                ('O', 'r_sns_3', 'N'): 10,
                ('O', 'r_sns_4', 'O'): 10,
                ('N', 'r_transfer_1', 'N'): 10,
                ('N', 'r_transfer_2', 'O'): 10,
                ('O', 'r_transfer_3', 'N'): 10,
                ('O', 'r_transfer_4', 'O'): 10}, 
               
               {('N', 'r_contact_1', 'N'): 10,
                ('N', 'r_contact_2', 'O'): 10,
                ('O', 'r_contact_3', 'N'): 10,
                ('O', 'r_contact_4', 'O'): 10,
                ('N', 'r_sns_1', 'N'): 10,
                ('N', 'r_sns_2', 'O'): 10,
                ('O', 'r_sns_3', 'N'): 10,
                ('O', 'r_sns_4', 'O'): 10,
                ('N', 'r_transfer_1', 'N'): 10,
                ('N', 'r_transfer_2', 'O'): 10,
                ('O', 'r_transfer_3', 'N'): 10,
                ('O', 'r_transfer_4', 'O'): 10}, ] # 两层模型，需要定义每一层聚合neighbor的数量

save_name = 'HGT_model_221530_jd_semi'
dst_dict = {"N": test_idx}
# model.load_state_dict(torch.load('best_'+save_name+'.pth'))
model.load_state_dict(torch.load(save_name+'.pth'))
# write_odps_table_name = 'ant_prot_dev_minghai_323064_inference_test'

pd_df = inference(model, g, "N", sample_list, dst_dict, args.batch_size, '', raw_label='test_label', project='ant_prot_dev')

### 评估结果 ###
pred = pd_df.sort_values(by='score1',ascending=False)
pred['pred_label'] = [1]*174+[0]*174


pred_acc = np.mean(pred[['label']].values == pred[['pred_label']].values)
print('pred_acc: ',pred_acc)
recall = sum(pred['label'][:175])/sum(pred['label'])
print('recall: ',recall)
precesion = sum(pred['label'][:175])/175
print('precesion: ',precesion)

strat to inference
2021-12-09 14:52:40
label>>>>> 348
logits>>>>> 348
pred_acc:  0.4285737878187343
inference succeed
2021-12-09 14:52:40
pred_acc:  0.5488505747126436
recall:  0.542713567839196
precesion:  0.6171428571428571


In [23]:
pred

id  label    score0    score1  pred_label
89     124.0    1.0  0.532668  0.467332           1
72     145.0    0.0  0.572361  0.427639           1
284  26398.0    1.0  0.580650  0.419350           1
126  27872.0    0.0  0.586955  0.413045           1
180  13164.0    1.0  0.590183  0.409817           1
..       ...    ...       ...       ...         ...
222  16235.0    0.0  0.704982  0.295018           0
226  19218.0    0.0  0.707052  0.292948           0
232  33845.0    1.0  0.710326  0.289674           0
178  38198.0    0.0  0.711147  0.288853           0
113  28070.0    0.0  0.716625  0.283375           0

[348 rows x 5 columns]